In [2]:
git add portfolio.ipynb

SyntaxError: invalid syntax (3757208924.py, line 1)

In [ ]:
git commit -m "Add portfolio notebook"

In [ ]:
git remote add origin https://github.com/<levolak>/<https://github.com/levolak/cartogis-portfolio.git>.git

In [ ]:
r